<a href="https://colab.research.google.com/github/MehulLuthra/Mashup/blob/main/102317146.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, render_template, request, jsonify, send_file
import os
import sys
import shutil
import subprocess
import tempfile
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText
import zipfile
import yt_dlp
import time
import uuid
import threading
import base64
import requests
from dotenv import load_dotenv

load_dotenv()

app = Flask(__name__, static_folder='.', static_url_path='')

MASHUP_STORAGE = {}
EMAIL_STATUS = {}

FFMPEG_PATH = os.environ.get('FFMPEG_PATH', '/opt/homebrew/bin/ffmpeg')
if not os.path.exists(FFMPEG_PATH):
    FFMPEG_PATH = 'ffmpeg'


def create_mashup(song_filenames, duration):
    temp_dir = tempfile.mkdtemp()
    output_file = os.path.join(temp_dir, "output.mp3")
    downloads_folder = os.path.join(os.path.dirname(__file__), "downloads")

    try:
        source_files = []
        for filename in song_filenames:
            path = os.path.join(downloads_folder, filename)
            if os.path.exists(path):
                source_files.append(path)

        if not source_files:
            raise Exception("No valid songs selected")

        temp_clips = []
        for idx, audio in enumerate(source_files):
            clip = os.path.join(temp_dir, f"clip_{idx}.mp3")
            cmd = [FFMPEG_PATH, '-i', audio, '-t', str(duration),
                   '-acodec', 'libmp3lame', '-y', clip]
            subprocess.run(cmd, stdout=subprocess.DEVNULL,
                           stderr=subprocess.DEVNULL, check=True)
            temp_clips.append(clip)

        concat_file = os.path.join(temp_dir, "concat_list.txt")
        with open(concat_file, 'w') as f:
            for clip in temp_clips:
                f.write(f"file '{clip}'\n")

        cmd = [FFMPEG_PATH, '-f', 'concat', '-safe', '0',
               '-i', concat_file, '-c', 'copy', '-y', output_file]
        subprocess.run(cmd, stdout=subprocess.DEVNULL,
                       stderr=subprocess.DEVNULL, check=True)

        return output_file, temp_dir

    except Exception as e:
        shutil.rmtree(temp_dir, ignore_errors=True)
        raise e


def send_email(to_email, attachment_path, songs_list):
    from_email = os.environ.get('MAIL_USERNAME')
    password = os.environ.get('MAIL_PASSWORD')

    if not from_email or not password:
        raise Exception("Email credentials not configured")

    msg = MIMEMultipart('alternative')
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = '🎵 Your Custom Mashup is Ready!'

    songs_html = "".join(
        [f"<li style='margin:6px 0'>{s}</li>" for s in songs_list])

    html_body = f"""
    <html>
    <body style="font-family: Arial; background:#f5f5f5; padding:30px;">
        <div style="background:#fff; padding:30px; border:2px solid #000;">
            <h2>Your Mashup is Ready 🎵</h2>
            <p>Songs included:</p>
            <ul>{songs_html}</ul>
            <p><strong>Attachment:</strong> mashup.zip</p>
            <hr>
            <p><strong>Made by Mehul Luthra</strong></p>
            <p>Roll No: 102317146</p>
        </div>
    </body>
    </html>
    """

    text_body = f"""
    Your Mashup is Ready!

    Songs:
    {chr(10).join(songs_list)}

    Made by Mehul Luthra
    Roll No: 102317146
    """

    msg.attach(MIMEText(text_body, 'plain'))
    msg.attach(MIMEText(html_body, 'html'))

    zip_path = attachment_path.replace('.mp3', '.zip')
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        zipf.write(attachment_path, 'custom_mashup.mp3')

    with open(zip_path, 'rb') as f:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(f.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',
                        'attachment; filename=mashup.zip')
        msg.attach(part)

    raise Exception("Mail sending disabled for safety")


@app.route('/')
def index():
    return send_file('index.html')


@app.route('/api/create-mashup', methods=['POST'])
def create_mashup_api():
    try:
        data = request.get_json()
        songs = data.get('songs', [])
        duration = int(data.get('duration', 20))
        email = data.get('email')

        output_file, temp_dir = create_mashup(songs, duration)
        mashup_id = str(uuid.uuid4())

        zip_path = output_file.replace('.mp3', '.zip')
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            zipf.write(output_file, 'custom_mashup.mp3')

        MASHUP_STORAGE[mashup_id] = {
            'zip_path': zip_path,
            'temp_dir': temp_dir
        }

        return jsonify({
            'message': 'Mashup created successfully!',
            'download_url': f'/download-mashup/{mashup_id}'
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/download-mashup/<mashup_id>')
def download_mashup(mashup_id):
    data = MASHUP_STORAGE.get(mashup_id)
    if not data:
        return jsonify({'error': 'Not found'}), 404
    return send_file(data['zip_path'], as_attachment=True)


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
